## Tecnológico de Costa Rica

### Autor: Jorge Andrés Brenes Alfaro

## Red mimetizadora



## 1. Bibliotecas

In [1]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Dense, Dropout, GRU, TimeDistributed

import warnings
warnings.filterwarnings('ignore')

## 2. Dataset 

In [2]:
Dir = os.listdir('/Users/jorge/Documents/TEC/TFG/Datos_Recolectados')

pwm = np.array([])
angle = np.array([])

for filename in Dir:
    files = pd.read_csv('/Users/jorge/Documents/TEC/TFG/Datos_Recolectados/'+filename)
    pwm = np.append(pwm,np.zeros(100))
    pwm = np.append(pwm,files.values[:,2])
    angle = np.append(angle,np.zeros(100))
    angle = np.append(angle,files.values[:,3])

X_train = []
Y_train = []
window = 100

for i in range(window,pwm.shape[0]):
    X_train.append(pwm[i-window:i])
    Y_train.append(angle[i])
X_train, Y_train = np.array(X_train), np.array(Y_train)


train_data, val_data, test_data = [],[],[]
train_label, val_label, test_label = [],[],[]

train_lenght = int(len(X_train)*3/5)
val_lenght = int(len(X_train)*4/5)

for i in X_train[:train_lenght]:
    train_data.append(i)
    train_label.append(i)

for i in X_train[train_lenght:val_lenght]:
    val_data.append(i)
    val_label.append(i)
    
for i in X_train[val_lenght:]:
    test_data.append(i)
    test_label.append(i)

train_data, val_data, test_data = np.array(train_data), np.array(val_data), np.array(test_data)
train_label, val_label, test_label = np.array(train_label), np.array(val_label), np.array(test_label)

print('El total de datos de entrenamiento es: ', len(train_data))
print('El total de datos de validación es: ', len(val_data))
print('El total de datos de prueba es: ', len(test_data))

train_data = np.reshape(train_data,(train_data.shape[0],train_data.shape[1],1))
val_data = np.reshape(val_data,(val_data.shape[0],val_data.shape[1],1))
test_data = np.reshape(test_data,(test_data.shape[0],test_data.shape[1],1))

El total de datos de entrenamiento es:  82911
El total de datos de validación es:  27637
El total de datos de prueba es:  27638


## 3. Neural Network

### 3.1 Model Creation

In [3]:
clear_session()

model = Sequential()
model.add(GRU(64, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.35))
model.add(GRU(64, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.35))
model.add(TimeDistributed(Dense(1))) # There is no difference between this and model.add(Dense(1))...
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mse','acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 100, 64)           12864     
_________________________________________________________________
dropout (Dropout)            (None, 100, 64)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 64)           24960     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 100, 1)            65        
Total params: 37,889
Trainable params: 37,889
Non-trainable params: 0
_________________________________________________________________


### 3.2 Model Training

In [4]:
history = model.fit(train_data, train_label,
                    epochs=500, batch_size=8,
                    validation_data = (val_data,val_label),
                    verbose=2)

#testPredict = model.predict(test_data)

Epoch 1/500
10364/10364 - 438s - loss: 6.3266e-05 - mse: 6.3266e-05 - acc: 0.2119 - val_loss: 2.1178e-06 - val_mse: 2.1178e-06 - val_acc: 0.4070
Epoch 2/500
10364/10364 - 496s - loss: 4.3730e-05 - mse: 4.3730e-05 - acc: 0.2119 - val_loss: 7.4353e-07 - val_mse: 7.4353e-07 - val_acc: 0.4070
Epoch 3/500


KeyboardInterrupt: 

### 3.3 Model Evaluate

In [ ]:
#loss, accuracy = model.evaluate(testX,output_test)

In [ ]:
joblib.dump(model, 'GRU_model.joblib')

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss)+1)
plt.figure()
plt.plot(epochs, loss,'b', label='Training loss')
plt.plot(epochs, val_loss,'r', label='Validation loss')
plt.title('Training and validation losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xscale('log')
plt.legend()
plt.show()